# DeepLandforms Inference YOLOv5
## Landforms Object Detection Inference and shapefile creation using YOLOv5
@ g.nodjoumi@jacobs-university.de

***This notebook is based on yolov5 repository is provided by ultralytics https://zenodo.org/record/4154370 that has been modified to adapt to landforms***


In [ ]:
%cd yolov5
import os
from adds.GenUtils import question, get_types, get_paths, make_folder, askPath, askFile, askInt, askFloat
from adds.DLUtils import get_train_cfg, link_dataset, get_model_cfg, get_args
from adds.inf2shp import det2gdf, copyfiles, get_world_file
import geopandas as gpd
import pandas as pd
import torch
from pyproj import CRS
from IPython.display import Image  # for displaying images

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
wts = askFile('Weights file: ')
source = askPath('Path to input images: ')
output = source+'/output'
img_size = askInt('Image size for inference: ') # depending on available VRAM, e.g. for 8GB VRAM max img_size is 6000
conf_thres = askFloat('Confidence threshold') # suggested is 0.5
home = os.getcwd()

### Import modules

### Create dictionary with all parameters
***edit if necessary***

In [ ]:
DST_CRS= CRS.from_wkt('PROJCRS["Equirectangular MARS",BASEGEOGCRS["GCS_MARS",DATUM["unnamed",ELLIPSOID["unnamed",3396036.8126024,0,LENGTHUNIT["metre",1,ID["EPSG",9001]]]],PRIMEM["Reference meridian",0,ANGLEUNIT["degree",0.0174532925199433,ID["EPSG",9122]]]],CONVERSION["unnamed",METHOD["Equidistant Cylindrical (Spherical)",ID["EPSG",1029]],PARAMETER["Latitude of 1st standard parallel",-5,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Longitude of natural origin",180,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["northing",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]')

In [ ]:
args_dict = {
    '--weights': wts,
    '--source':source,
    '--img-size':img_size,
    '--conf-thres':conf_thres,
    '--iou-thres':0.4,
    '--device':'',
    '--view-img': '',
    '--save-txt': True,
    '--save-conf':True,
    '--classes': '',
    '--agnostic_nms':'',
    '--augment':'',
    '--update':'',
    '--project':source,
    '--name':'output',
    '--exist-ok':''}

In [ ]:
arv = get_args(args_dict)

In [ ]:
os.chdir(home)
!python detect.py $arv

## Convert detection to shapefile

**Create list of detections**

In [ ]:
detections = get_paths(output+'/labels', 'txt')

**Create final dataframe**

In [ ]:
detections_paths = [source+'output/labels/'+file for file in detections]

In [ ]:
geodataframes = det2gdf(detections_paths, 'tiff', source)

In [ ]:
try:
    geoshapes.crs
except:
    geoshapes = geodataframes[0]

In [ ]:
init_crs = geodataframes[0].crs
for geo in geodataframes:
    if geo.crs != init_crs:
        geo = geo.to_crs(init_crs)
#    geoshapes.append(geo, ingore_index=True)
    geoshapes = gpd.GeoDataFrame(pd.concat([geoshapes,geo],ignore_index=True),crs=init_crs)

**Save to shapefile**

In [ ]:
fname = make_folder(output, 'Dataframe')
geoshapes.to_file(fname+'/Inferred_landforms.gpkg', driver='GPKG', crs=init_crs) 